# Ackerberg sampling and onwards

In this notebook, we simulate out the games, create importance weights, and do a lot of other juicy stuff to get ready for estimation. There is a lot of looping and programming here, so this could get long, but let's try to keep things as transparent as possible. 

We should describe the essence of Ackerberg sampling here as well. 

The first steps are basically the same as usual:

In [1]:
import ipystata
import os
os.getcwd()

'C:\\Users\\Administrator\\Documents\\github\\newshour'

In [2]:
%%stata
cd C:\Users\Administrator\Documents\GitHub\NewsHour
clear all
use AveragedDataDyno.dta
set more off


C:\Users\Administrator\Documents\GitHub\NewsHour


The first thing we will do is generate (if it is not already there) a marker for who is in the game and who is not - **note**: this is perhaps where branchings of our work will begin in the future. So:

In [3]:
%%stata
capture gen game= (lnews | otherl) & lnpps!=.
sort market stationid mt timeslot

Note that the above code reflects the idea that only those stations broadcasting local news or other local programming (entertainment programming) in the actual paper are "making decisions" at any given time. Also, we need to have a price variable for them as well. 

The usual next step is to pull everything we need into `Mata`, while also generating any additional variables that we might need. 

In [4]:
%%stata
mata:
    st_view(lnews=.,.,"lnews")
    st_view(nnews=.,.,"nnews")
    st_view(otherl=.,.,"otherl")
    st_view(otherc=.,.,"otherc")
    st_view(si=.,.,"si")
    st_view(mt=.,.,"mt")
    st_view(id=.,.,"stationid")
    st_view(M=.,.,"market")
    st_view(time=.,.,"timeslot")
    st_view(game=.,.,"game")
    st_view(lnpps=.,.,"lnpps")
    st_view(pop=.,.,"ACS_HH") 
    st_view(l_ACS_HH=.,.,"l_ACS_HH")
    st_view(lnewsn=.,.,"lnewsn")
    st_view(otherln=.,.,"otherln")
    st_view(nnewsn=.,.,"nnewsn")
    st_view(othercn=.,.,"othercn")
    st_view(dln=.,.,"dln")

    lnviewn=ln(pop:*si)
    m=panelsetup(M,1)
end


------------------------------------------------- mata (type end to exit) ----------------------------------------------------------------
:     st_view(lnews=.,.,"lnews")

:     st_view(nnews=.,.,"nnews")

:     st_view(otherl=.,.,"otherl")

:     st_view(otherc=.,.,"otherc")

:     st_view(si=.,.,"si")

:     st_view(mt=.,.,"mt")

:     st_view(id=.,.,"stationid")

:     st_view(M=.,.,"market")

:     st_view(time=.,.,"timeslot")

:     st_view(game=.,.,"game")

:     st_view(lnpps=.,.,"lnpps")

:     st_view(pop=.,.,"ACS_HH") 

:     st_view(l_ACS_HH=.,.,"l_ACS_HH")

:     st_view(lnewsn=.,.,"lnewsn")

:     st_view(otherln=.,.,"otherln")

:     st_view(nnewsn=.,.,"nnewsn")

:     st_view(othercn=.,.,"othercn")

:     st_view(dln=.,.,"dln")

:     m=panelsetup(M,1)

: end


Pull in the starting values and get them all set up. Note this requires a little bit of rearranging and some transformation to suit the way we entered things into the likelihood. We also set the number of draws at 20 - while this might seem low, it is hard to do more because the problem is so giant!

In [5]:
%%stata
mata:
    mata matuse DynoStarts23

    bo[,1::4]=bo[,1::4]:^2
    bo=bo[rows(bo),]


    betaDynoStart=bo[(5..23,27)]

    sdstav=exp(bo[24])
    sdmarv=exp(bo[25])
    sdmodv=exp(bo[26])
    
    draws=20
end:


------------------------------------------------- mata (type end to exit) ----------------------------------------------------------------
:     mata matuse DynoStarts23
(loading arate[1,27], bo[1,27], drawsbo[100,27], vals[100,1])

:     bo=bo[rows(bo),]

:     betaDynoStart=bo[(5..23,27)]

:     sdmarv=exp(bo[25])

:     sdmodv=exp(bo[26])

:     
:     draws=20

: end:


In a previous workbook, we did a big reshape and some associated calculations to get the data in mata form, and in reshaped form. That is, so we have stations as observations, with each row representing a share, price, decision, etc. of dimension 6 - the number of time periods. 

This is commonly referred to as wide format in Stata, which, for some reason, I decided to mark by suffixing things with the `Long`. To keep things short, I gathered all this code (it is redundant) into a script called `MataReshape.do`. The resulting variables are defined exactly as they were in a previous workbook. 

In [6]:
%%stata
do MataReshape.do


------------------------------------------------- mata (type end to exit) ----------------------------------------------------------------
:         
:         timeslots=6

:         lnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         nnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         otherlLong=J(rows(uniqrows(id)),timeslots,.)

:         othercLong=J(rows(uniqrows(id)),timeslots,.)

:         slnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         snnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         sotherlLong=J(rows(uniqrows(id)),timeslots,.)

:         sothercLong=J(rows(uniqrows(id)),timeslots,.)

:         gameLong=J(rows(uniqrows(id)),timeslots,.)

:         popLong=J(rows(uniqrows(id)),timeslots,.)  /* Not necessary but for convenience and completeness */

:         l_ACS_HHLong=J(rows(uniqrows(id)),timeslots,.)

:         
:         siLong=J(rows(uniqrows(id)),timeslots,.)

:         lnewsLongLag=J(rows(uniqrows(id)),timeslots,.)

:         nnewsLongLag=J(rows(uniq

That having been done, we now have to recover actual viewership errors, given our model parameters. More correctly, what we really need is the utility index for viewership. This is pretty easy to calculate given our model estimates. If we have (ignoring subscripts, etc. just to give the basic idea):

$$
\ln(s_i)-\ln(s_0) = \mu \ln(s_{ig})+U_i
$$

where $s_{ig}$ is the share of $i$ in $i$'s programming group. 

We can get $U_i=X\beta_v+\epsilon$ by doing:

$$
U_i = \ln(s_i)-\ln(s_0)-\mu \ln (s_{ig})
$$

In St(M)ata, we calculate the within group share (even though we have it - all these redundancies are good for checking things are working), and then calculate this linear viewership term:

In [7]:
%%stata
mata:
    swgLong=siLong:/(slnewsLong:*lnewsLong:+
        snnewsLong:*nnewsLong:+
        sotherlLong:*otherlLong:+
        sothercLong:*othercLong)
    
    lnswgLong=ln(swgLong)   

    dln=ln(siLong):-ln(1:-(slnewsLong:+snnewsLong:+sotherlLong:+sothercLong))    
    
    XBVngLong=dln-bo[1]*lnswgLong:*lnewsLong-bo[2]*lnswgLong:*otherlLong-
        bo[3]*lnswgLong:*nnewsLong-bo[4]*lnswgLong:*othercLong
end


------------------------------------------------- mata (type end to exit) ----------------------------------------------------------------
:     swgLong=siLong:/(slnewsLong:*lnewsLong:+
>         snnewsLong:*nnewsLong:+
>         sotherlLong:*otherlLong:+
>         sothercLong:*othercLong)

:     
:     lnswgLong=ln(swgLong)   

:     
:     XBVngLong=dln-bo[1]*lnswgLong:*lnewsLong-bo[2]*lnswgLong:*otherlLong-
>         bo[3]*lnswgLong:*nnewsLong-bo[4]*lnswgLong:*othercLong

: end


In [8]:
%%stata
mata: XBVngLong[1::10,]
mata: hasmissing(XBVngLong)


                   1              2              3              4              5              6
     +-------------------------------------------------------------------------------------------+
   1 |  -8.985634105   -8.392608246   -8.731478588   -8.433964942   -6.958238826   -6.845453225  |
   2 |  -8.151051388   -8.357332086   -8.688881175   -8.615198359   -7.966533755   -7.965828642  |
   3 |  -8.235545882   -8.282268759   -8.344499022    -8.46101344   -8.280310383   -8.178011364  |
   4 |  -9.450145217   -9.174906813   -9.430928685   -8.853047388   -9.087496771   -9.138757042  |
   5 |  -9.178007157   -9.091459258    -8.41704476   -8.299877452   -8.366149049   -8.268955677  |
   6 |  -8.636742049   -8.510143242   -9.197289446   -8.994011156   -7.608889701     -7.5129947  |
   7 |  -8.742925026   -8.627547681   -8.729837754   -8.358080888   -8.013807285   -7.947865051  |
   8 |  -8.185559242   -7.863383017   -7.539083529   -7.445749092   -7.030719852   -6.766605178  |
   9 |  -7.7

These seem about right. so, now what we can do is start drawing random effects and all that. To do this, we can write a little function called `paneldraw()` which draws common values across a user-specified panel identifier. I've written this function so that it works with the id only, and doesn't require the data to be sorted. 

In [9]:
%%stata
mata: 
    real matrix paneldraw(real matrix id, real scalar draws)
    {
        rowsToDraw=uniqrows(id)

        U=J(rows(id),draws,.)
        for (i=1;i<=rows(rowsToDraw);i++) {
            obs=mm_which(rowsToDraw[i]:==id)
            D=runiform(1,draws)
            U[obs,]=J(rows(obs),1,D)
        }
        return(U)
    }
end


------------------------------------------------- mata (type end to exit) ----------------------------------------------------------------
:     real matrix paneldraw(real matrix id, real scalar draws)
>     {
>         rowsToDraw=uniqrows(id)
> 
>         U=J(rows(id),draws,.)
>         for (i=1;i<=rows(rowsToDraw);i++) {
>             obs=mm_which(rowsToDraw[i]:==id)
>             D=runiform(1,draws)
>             U[obs,]=J(rows(obs),1,D)
>         }
>         return(U)
>     }

: end


Note that this just provides uniform random variates with common values across panel variables. We just use our estimated parameters and the usual inverse-normal transformation to get the actual values. We then reshape these into a useable form and call the results `UvsLong` and `UvmtLong` - the former being the station random effect, the latter being the market random effect. 

As the last part of the code block, we draw a purely idiosynchratic error term. 

Note that the stata package `moremata` is required for this step.

In [10]:
%%stata
mata:
    iev1=paneldraw(id,draws)
    iev2=paneldraw(mt,draws)
    iev1=sdstav*invnormal(iev1)
    iev2=sdmarv*invnormal(iev2)

    UvsLong=J(rows(lnewsLong),0,0)
    UvmtLong=J(rows(lnewsLong),0,0)
    
    for (i=1;i<=draws;i++) {
        UvsLong=UvsLong,colshape(iev1[,i],timeslots)
        UvmtLong=UvmtLong,colshape(iev2[,i],timeslots)
    }

    UvmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodv
end



------------------------------------------------- mata (type end to exit) ----------------------------------------------------------------
:     iev1=paneldraw(id,draws)

:     iev2=paneldraw(mt,draws)

:     iev1=sdstav*invnormal(iev1)

:     iev2=sdmarv*invnormal(iev2)

:     UvmtLong=J(rows(lnewsLong),0,0)

:     
:     for (i=1;i<=draws;i++) {
>         UvsLong=UvsLong,colshape(iev1[,i],timeslots)
>         UvmtLong=UvmtLong,colshape(iev2[,i],timeslots)
>     }

: end
